Purpose:
Retrieve full list of UK stations from http://orr.gov.uk/statistics/published-stats/station-usage-estimates

Create tables in the database and add data

Programmer:  Robin Linacre

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [4]:
sql = """
DROP TABLE IF EXISTS tt_h.all_stations;
DROP TABLE IF EXISTS tt_h.inner_london;
DROP TABLE IF EXISTS tt_h.m25;
"""
cursor.execute(sql)
conn.commit()

In [5]:
stations_df = pd.read_excel("http://www.orr.gov.uk/__data/assets/excel_doc/0020/23357/estimates-of-station-usage-2015-16.xlsx", 2)

In [6]:
headers = [h.lower() for h in list(stations_df.columns)]
headers = [h.replace(" ", "_").replace("(", "").replace(")","") for h in headers]
stations_df.columns = headers
stations_df.head()
stations_df = stations_df[pd.notnull(stations_df["nlc"])]

In [7]:
stations_df["nlc"] = stations_df["nlc"].astype(int)

In [8]:
# Now write out to postgres

stations_df.to_sql("all_stations", engine, schema="tt_h", if_exists="replace", index=False)

In [9]:
# Create geometry column for the points including a spatial index for efficient querying

sql = """
SELECT AddGeometryColumn ('tt_h', 'all_stations', 'geom', 27700, 'POINT', 2);
UPDATE tt_h.all_stations SET geom = ST_GeomFromText('POINT(' || os_grid_easting || ' ' || os_grid_northing || ')', 27700 );
CREATE INDEX idx_geom_all_stations_points ON tt_h.all_stations USING gist(geom);
"""
cursor.execute(sql)
conn.commit()

In [10]:
#Make a lat and lng column

sql = """
ALTER TABLE tt_h.all_stations ADD lat float, ADD lng float, ADD icscode text, 
ADD icscode_status text, ADD tfl_request text, ADD tfl_response json, ADD tfl_message  text;
UPDATE tt_h.all_stations SET
    lng = ST_X(ST_TRANSFORM(geom, 4326)),
    lat = ST_Y(ST_TRANSFORM(geom,4326));
ALTER TABLE tt_h.all_stations  ADD PRIMARY KEY (nlc);
""" 
cursor.execute(sql)
conn.commit()

In [11]:
%%bash
shp2pgsql -I -s 27700 shapefiles/inner_london.shp tt_h.inner_london | psql -d postgres
shp2pgsql -I -s 27700 shapefiles/m25.shp tt_h.m25 | psql -d postgres

SET
SET
BEGIN
CREATE TABLE
ALTER TABLE
                      addgeometrycolumn                      
-------------------------------------------------------------
 tt_h.inner_london.geom SRID:27700 TYPE:MULTIPOLYGON DIMS:2 
(1 row)

INSERT 0 1
CREATE INDEX
COMMIT
ANALYZE
SET
SET
BEGIN
CREATE TABLE
ALTER TABLE
                 addgeometrycolumn                  
----------------------------------------------------
 tt_h.m25.geom SRID:27700 TYPE:MULTIPOLYGON DIMS:2 
(1 row)

INSERT 0 1
CREATE INDEX
COMMIT
ANALYZE


Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]
Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]


In [12]:
sql = """
ALTER TABLE tt_h.all_stations ADD in_inner_london boolean;
ALTER TABLE tt_h.all_stations ADD in_m25 boolean;
UPDATE tt_h.all_stations SET in_inner_london = null;
UPDATE tt_h.all_stations SET in_m25 = null;
"""
cursor.execute(sql)
conn.commit()

In [5]:

sql = """
(SELECT s.nlc
from tt_h.all_stations as s, tt_h.inner_london as ie
where ST_Contains(ie.geom, s.geom))
"""
nlcs = pd.read_sql(sql, conn)
nlcs = list(nlcs.loc[:, "nlc"])

all_stations = Automapped_Base.classes.all_stations
stations = session.query(all_stations)
for station in stations:
    station.in_inner_london = (station.nlc in nlcs)
    session.add(station)
session.commit()


sql = """
(SELECT s.nlc
from tt_h.all_stations as s, tt_h.m25 as ie
where ST_Contains(ie.geom, s.geom))
"""
nlcs = pd.read_sql(sql, conn)
nlcs = list(nlcs.loc[:, "nlc"])

all_stations = Automapped_Base.classes.all_stations
stations = session.query(all_stations)
for station in stations:
    station.in_m25 = (station.nlc in nlcs)
    session.add(station)
session.commit()

In [3]:
sql = """
ALTER TABLE tt_h.all_stations ADD in_sprawl boolean;
UPDATE tt_h.all_stations SET in_sprawl = null;
"""
cursor.execute(sql)
conn.commit()



ProgrammingError: column "in_sprawl" of relation "all_stations" already exists


In [4]:
sql = """
(SELECT s.nlc
from tt_h.all_stations as s, (select st_buffer(geom, -5000) as geom from tt_h.m25) as ie
where ST_Contains(ie.geom, s.geom))
"""
nlcs = pd.read_sql(sql, conn)
nlcs = list(nlcs.loc[:, "nlc"])

all_stations = Automapped_Base.classes.all_stations
stations = session.query(all_stations)
for station in stations:
    station.in_sprawl = (station.nlc in nlcs)
    session.add(station)
session.commit()

